<a href="https://colab.research.google.com/github/KarinaKatke/HiRiD/blob/main/getting_started.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import glob
import math
import numpy as np
import os
import pandas as pd
import re
import shutil
import sys

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data_path = '/content/drive/My Drive/HiRID/raw_stage'

In [5]:
# Extract Raw Data
import tarfile

file_path = '/content/drive/My Drive/HiRID/raw_stage/observation_tables_parquet.tar.gz'
extract_path = '/content/observation_tables'

with tarfile.open(file_path, 'r:gz') as tar:
    tar.extractall(path=extract_path)

In [6]:
import os

extracted_files = os.listdir(extract_path)
print(extracted_files)

['observation_tables']


In [7]:
observation_table_path = os.path.join(extract_path, 'observation_tables')

In [8]:
# Listing of all parquet files
all_parquet_files = []
for root, dirs, files in os.walk(observation_table_path):
    for file in files:
        if file.endswith('.parquet'):
            all_parquet_files.append(os.path.join(root, file))

print(all_parquet_files)

['/content/observation_tables/observation_tables/parquet/part-1.parquet', '/content/observation_tables/observation_tables/parquet/part-222.parquet', '/content/observation_tables/observation_tables/parquet/part-144.parquet', '/content/observation_tables/observation_tables/parquet/part-236.parquet', '/content/observation_tables/observation_tables/parquet/part-84.parquet', '/content/observation_tables/observation_tables/parquet/part-201.parquet', '/content/observation_tables/observation_tables/parquet/part-219.parquet', '/content/observation_tables/observation_tables/parquet/part-123.parquet', '/content/observation_tables/observation_tables/parquet/part-80.parquet', '/content/observation_tables/observation_tables/parquet/part-129.parquet', '/content/observation_tables/observation_tables/parquet/part-197.parquet', '/content/observation_tables/observation_tables/parquet/part-34.parquet', '/content/observation_tables/observation_tables/parquet/part-183.parquet', '/content/observation_tables/

In [ ]:
#for parquet_file in all_parquet_files:
 #   df = pd.read_parquet(parquet_file)
  #  print(f"Reading file: {parquet_file}")
   # print(df.columns)

## Reading using Pandas
Required python packages are `pandas` and `pyarrow`

### Reading a part

In [ ]:
part = 4

In [ ]:
# Pandas-Optionen setzen für bessere Anzeige
pd.set_option("display.max_columns", None)        # alle Spalten anzeigen
pd.set_option("display.expand_frame_repr", False) # keine Umbrüche


In [ ]:
file_path = os.path.join(
    'observation_tables', 'observation_tables', 'parquet', f'part-{part}.parquet'
)

In [ ]:
# Anzeigen der Spaltennmane des parquet-files

df_part = pd.read_parquet(file_path)
print(f"Datei geladen: {file_path}")
print(df_part.columns)


Datei geladen: observation_tables/observation_tables/parquet/part-4.parquet
Index(['datetime', 'entertime', 'patientid', 'status', 'stringvalue', 'type',
       'value', 'variableid'],
      dtype='object')


In [ ]:
df_part['value'].count() #zählt wie viele Werte in der Spalte 'value' nicht leer sind, also wie viele Werte existieren

np.int64(2412421)

In [ ]:
# die ersten 10 Zeilen anzeigen lassen des parquet files

print(df_part.head(10))

                 datetime               entertime  patientid  status stringvalue type  value  variableid
0 2111-04-07 17:50:00.000 2111-04-07 18:02:16.320         65       8        None        90.0    10000400
1 2111-04-07 17:50:00.000 2111-04-07 18:02:16.333         65       8        None       175.0    10000450
2 2111-04-07 18:00:00.000 2111-04-07 19:46:35.530         65       8                     0.0    30005010
3 2111-04-07 18:00:00.000 2111-04-07 19:46:36.140         65       8                     0.0    30005075
4 2111-04-07 18:02:54.220 2111-04-07 18:02:58.723         65       4                   126.0         200
5 2111-04-07 18:02:55.070 2111-04-07 18:02:58.803         65       4                    88.0        4000
6 2111-04-07 18:03:28.100 2111-04-07 18:03:40.316         65      16                    88.0        4000
7 2111-04-07 18:03:28.550 2111-04-07 18:03:40.363         65      16                   127.0         200
8 2111-04-07 18:04:34.850 2111-04-07 18:05:38.986      

# Betrachtete Value


In [ ]:
# Die gewünschte variableid setzen
variableid = 300


# Wie viele Einträge hat eine bestimmte Variable?

In [ ]:
total_count = 0
variableid = 3845

In [ ]:
# zählen wie viel data wir zu einer spezifischen value haben, in diesem parquet file

gefilterte_values = df_part[df_part['variableid'] == variableid]['value']
total_count = gefilterte_values.notna().sum()

print(f"\nGesamtanzahl nicht-NaN-Werte für VariableID {variableid}: {total_count}")
print(gefilterte_values)


Gesamtanzahl nicht-NaN-Werte für VariableID 3845: 16068
11806      13.0
11902      13.0
12078      13.0
12213      13.0
12334      13.0
           ... 
2392581     1.0
2392585     1.0
2392589     1.0
2392593     1.0
2392597     1.0
Name: value, Length: 16068, dtype: float32


In [ ]:
# durch alle Parquet Files gehen und zählen:

# Zähler initialisieren
gesamt_count = 0

# Durchlaufen aller Parquet-Dateien
for file_path in all_parquet_files:
    print(f"Lade Datei: {file_path}")
    df = pd.read_parquet(file_path)

    # Filtern nach der gewünschten variableid
    gefilterte_values = df[df['variableid'] == variableid]['value']

    # Zählen nicht-NaN-Werte in der Spalte "value"
    count = gefilterte_values.notna().sum()
    gesamt_count += count

    print(f"{count} Werte in Datei gefunden.")

print(f"\n Gesamtanzahl nicht-NaN-Werte für VariableID {variableid}: {gesamt_count}")

Lade Datei: /content/observation_tables/observation_tables/parquet/part-1.parquet
24313 Werte in Datei gefunden.
Lade Datei: /content/observation_tables/observation_tables/parquet/part-222.parquet
15504 Werte in Datei gefunden.
Lade Datei: /content/observation_tables/observation_tables/parquet/part-144.parquet
25512 Werte in Datei gefunden.
Lade Datei: /content/observation_tables/observation_tables/parquet/part-236.parquet
24017 Werte in Datei gefunden.
Lade Datei: /content/observation_tables/observation_tables/parquet/part-84.parquet
16467 Werte in Datei gefunden.
Lade Datei: /content/observation_tables/observation_tables/parquet/part-201.parquet
27031 Werte in Datei gefunden.
Lade Datei: /content/observation_tables/observation_tables/parquet/part-219.parquet
29674 Werte in Datei gefunden.
Lade Datei: /content/observation_tables/observation_tables/parquet/part-123.parquet
23439 Werte in Datei gefunden.
Lade Datei: /content/observation_tables/observation_tables/parquet/part-80.parquet


# Tabelle nur mit einer variableid

In [ ]:
variableid = 200

In [ ]:
# Filter anwenden
filtered_df = df_part[df_part["variableid"] == variableid]

# Nach Zeit sortieren, falls vorhanden
if "timestamp" in filtered_df.columns:
    filtered_df = filtered_df.sort_values("timestamp")

# Ausgabe
print(filtered_df.head(20))

                   datetime               entertime  patientid  status stringvalue type  value  variableid
4   2111-04-07 18:02:54.220 2111-04-07 18:02:58.723         65       4                   126.0         200
7   2111-04-07 18:03:28.550 2111-04-07 18:03:40.363         65      16                   127.0         200
9   2111-04-07 18:04:35.320 2111-04-07 18:05:39.033         65      16                   126.0         200
16  2111-04-07 18:06:49.040 2111-04-07 18:07:38.220         65      16                   127.0         200
21  2111-04-07 18:09:35.080 2111-04-07 18:09:37.250         65      16                   128.0         200
26  2111-04-07 18:10:09.280 2111-04-07 18:11:35.966         65      16                   124.0         200
28  2111-04-07 18:11:48.860 2111-04-07 18:13:34.730         65      16                   126.0         200
32  2111-04-07 18:15:07.850 2111-04-07 18:15:33.746         65      16                   123.0         200
33  2111-04-07 18:16:49.380 2111-04-0

# Label und Features

In [12]:
import pandas as pd
import pyarrow.parquet as pq
import os
from datetime import timedelta

# IDs für dein Projekt
VAR_IDS = {
    "TidalVolumeSetting": [2400],  # Label
    "PEEP": [2600, 2610],
    "FiO2": [2010],
    "PaCO2": [2001200],
    "pH": [20000300],
    "SaO2": [20000800],
    "RespRate": [300, 310, 5685],
    "VentMode": [3845],
    "Weight": [10000480]
}

# Zeitfenster (z. B. 60 Minuten vor Zielzeitpunkt)
window_minutes = 60

# Ergebnisse speichern
feature_rows = []

# Zähler für Debugging
processed_files_count = 0
total_patient_ids = 0
total_tv_labels = 0
rows_added_to_features = 0

# Durchlaufe alle parquet-Dateien
# Verwenden Sie die Variable observation_table_path, die den Pfad zum extrahierten Verzeichnis enthält
print(f"Suche nach Parquet-Dateien in: {observation_table_path}")
file_list = os.listdir(observation_table_path)
print(f"Gefundene Einträge im Verzeichnis: {len(file_list)}")


for part_path in file_list:
    # Konstruiere den vollständigen Pfad zur Datei
    full_file_path = os.path.join(observation_table_path, part_path)

    # Stellen Sie sicher, dass es sich um eine Datei handelt, bevor Sie versuchen, sie zu lesen
    if not os.path.isfile(full_file_path) or not full_file_path.endswith(".parquet"):
        print(f"Ignoriere: {full_file_path} (Keine Parquet-Datei)")
        continue

    print(f"Verarbeite Datei: {full_file_path}")
    processed_files_count += 1

    try:
        # Lesen Sie die Parquet-Datei
        df = pd.read_parquet(full_file_path)
        print(f"  Datei erfolgreich geladen. Zeilen: {len(df)}")

        if df.empty:
            print(f"  Datei ist leer.")
            continue

        # Filtere relevante Variablen
        relevant_ids = [vid for ids in VAR_IDS.values() for vid in ids]
        df_relevant = df[df['variableid'].isin(relevant_ids)].copy()
        print(f"  Zeilen nach Filtern nach relevanten VariableIDs ({len(relevant_ids)} IDs): {len(df_relevant)}")

        if df_relevant.empty:
             print(f"  Keine relevanten VariableIDs in dieser Datei gefunden.")
             continue

        # Zeiteinheit
        try:
            df_relevant['timestamp'] = pd.to_datetime(df_relevant['timestamp'])
            print(f"  Zeitstempel erfolgreich in datetime konvertiert.")
        except Exception as e:
            print(f"  FEHLER bei Zeitstempel-Konvertierung in {full_file_path}: {e}")
            continue


        # Nach Patientengruppierung
        patient_groups = df_relevant.groupby('patientid')
        print(f"  Gefundene PatientIDs: {len(patient_groups)}")
        total_patient_ids += len(patient_groups)

        for patient_id, patient_df in patient_groups:
            patient_df = patient_df.sort_values('timestamp')

            # Label-Zeitpunkte (wann immer Tidal Volume Setting vorhanden ist)
            tv_df = patient_df[patient_df['variableid'].isin(VAR_IDS['TidalVolumeSetting'])]
            print(f"    Patient {patient_id}: Gefundene TidalVolumeSetting-Zeitpunkte: {len(tv_df)}")
            total_tv_labels += len(tv_df)

            for _, row in tv_df.iterrows():
                t_label = row['timestamp']
                y = row['value']  # Label-Wert

                # Alle Features im Zeitfenster [t_label - 60 min, t_label]
                t_start = t_label - timedelta(minutes=window_minutes)
                df_window = patient_df[(patient_df['timestamp'] >= t_start) & (patient_df['timestamp'] <= t_label)].copy() # .copy() um SettingWithCopyWarning zu vermeiden
                #print(f"      Label-Zeitpunkt {t_label}: Zeilen im Fenster [{t_start}, {t_label}]: {len(df_window)}")

                features = {'patientid': patient_id, 'label_time': t_label, 'TidalVolumeSetting': y}

                # Zähler für Features im Fenster
                features_found_in_window = 0

                for feat_name, ids in VAR_IDS.items():
                    if feat_name == "TidalVolumeSetting":
                        continue
                    feat_values = df_window[df_window['variableid'].isin(ids)]
                    if not feat_values.empty:
                        # Nehme den letzten bekannten Wert
                        features[feat_name] = feat_values.sort_values('timestamp').iloc[-1]['value']
                        features_found_in_window += 1
                    else:
                        features[feat_name] = None  # NaN

                # Nur hinzufügen, wenn mindestens ein Feature im Fenster gefunden wurde (optional, je nach Anforderung)
                # if features_found_in_window > 0:
                feature_rows.append(features)
                rows_added_to_features += 1
                # else:
                #     print(f"      Label-Zeitpunkt {t_label}: Keine Features im Zeitfenster gefunden.")


    except Exception as e:
        print(f"  FEHLER bei Verarbeitung von Datei {full_file_path}: {e}")


print("\n--- Verarbeitung abgeschlossen ---")
print(f"Insgesamt verarbeitete Parquet-Dateien: {processed_files_count}")
print(f"Insgesamt gefundene eindeutige PatientIDs (in relevanten Daten): {total_patient_ids}")
print(f"Insgesamt gefundene TidalVolumeSetting-Zeitpunkte: {total_tv_labels}")
print(f"Insgesamt hinzugefügte Zeilen zum Feature-DataFrame: {rows_added_to_features}")

# In DataFrame umwandeln
df_features = pd.DataFrame(feature_rows)

# Zeige Beispiel
print("\nBeispiel-Features:")
print(df_features.head())

print(f"\nEndgültige Größe des Feature-DataFrames: {len(df_features)}")

Suche nach Parquet-Dateien in: /content/observation_tables/observation_tables
Gefundene Einträge im Verzeichnis: 2
Ignoriere: /content/observation_tables/observation_tables/observation_tables_index.csv (Keine Parquet-Datei)
Ignoriere: /content/observation_tables/observation_tables/parquet (Keine Parquet-Datei)

--- Verarbeitung abgeschlossen ---
Insgesamt verarbeitete Parquet-Dateien: 0
Insgesamt gefundene eindeutige PatientIDs (in relevanten Daten): 0
Insgesamt gefundene TidalVolumeSetting-Zeitpunkte: 0
Insgesamt hinzugefügte Zeilen zum Feature-DataFrame: 0

Beispiel-Features:
Empty DataFrame
Columns: []
Index: []

Endgültige Größe des Feature-DataFrames: 0
